In [1]:
!pip install html5lib beautifulsoup4
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd

In [2]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium.webdriver.common.keys import Keys

In [27]:
import numpy as np

In [170]:
# 인코딩이 euc-kr입니다. 저걸 지정하지 않으면 csv가 안 열려요!
# idol_list_withid를 불러오셔야 합니다. 멜론 url에 가수를 찾기 위한 id입니다. 

idol_list = pd.read_csv('idol_list_withid.csv', encoding='euc-kr')
idol_list.head()

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id
0,서태지와 아이들,SeoTaiji And Boys,3,서태지,남,요요 기획,100161.0
1,더 블루,The Blue,2,손지창,남,SM 엔터테인먼트,180144.0
2,잼,Zam,5,조진수,혼,DSP 미디어,225786.0
3,에스오에스,S.O.S,4,NaN,여,삼포니 레코드,101645.0
4,솔리드,Solid,3,정재윤,남,비앤비,100037.0


In [171]:
idol_list.shape

(339, 7)

In [172]:
# 멜론에 정보가 없어서 아예 페이지가 없었던 가수들이 몇몇 있습니다.
# 이런 경우 id가 공란이기에 수집이 불가합니다. 모두 드랍하겠습니다. 

idol_list = idol_list[np.isfinite(idol_list['idol_id'])]

In [173]:
idol_list.shape

(333, 7)

In [175]:
# id가 flaot으로 저장되어 있습니다. int로 바꿔주고 저장하겠습니다. 
idol_list['idol_id'] = idol_list['idol_id'].astype(int)
idol_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333 entries, 0 to 338
Data columns (total 7 columns):
artist         333 non-null object
artist_eng     332 non-null object
member_num     333 non-null object
leader_name    276 non-null object
gender         332 non-null object
agency         314 non-null object
idol_id        333 non-null int32
dtypes: int32(1), object(6)
memory usage: 19.5+ KB


In [106]:
# 수집을 위한 빈 리스트 생성

title = []
album = []
release_date = []
song_genre = []
like = []
lyrics = []
creator = []
artist = []
artist_m = []
debut_y = []
debut_t = []
agency = []
is_title = []

In [177]:
# 곡마다 가수 이름을 붙여주어야 하기에 가수 이름 리스트로 받아오기
names = idol_list.artist
names[0]

'비오케이'

In [178]:
# 크롤러가 링크를 찾을 때 매번 바꿔주는 값. for문에 사용됩니다. 
ids = idol_list.idol_id
ids[0]

1050192

In [109]:
# tests = ['861430', '789331']

## 크롤러

In [110]:
driver = webdriver.Chrome('C:/Users/pje17/Desktop/Lyrics/Crawlling/chromedriver')
driver.get("https://www.melon.com")

In [179]:
# 곡 정보를 찾아들어가기 위한 url을 분해해서 변수 지정해놓은 모습입니다. 
# l3 뒤에 붙는 숫자가 멜론 서버에 저장된 해당 가수의 해당 곡 번호입니다. 
# 레드벨벳의 32번째 곡은 뒤에 숫자가 32인 경우 곡 리스트에 첫번째로 보여집니다. 
# 즉 url을 바꿔가되 항상 리스트의 첫곡의 정보를 가져오면 모든 곡을 돌 수 있습니다. 

l1 = 'https://www.melon.com/artist/song.htm?artistId='
# idol_id
l2 = '#params%5BlistType%5D=A&params%5BorderBy%5D=ISSUE_DATE&params%5BartistId%5D='
# idol_id
l3 = '&po=pageObj&startIndex='
# song number
l1 + 'idol_id' + l2 + 'idol_id' + l3 + number

# 가수 이름을 넣기 위한 변수
i = 0

for id in ids:
    
    n = 1
    while True:
                 
        #sleep(3)
        # 해당 가수의 곡 리스트 n번째로 가기
        
        driver.get(l1+str(id)+l2+str(id)+l3+str(n))    
        sleep(2)
        
        # 타이틀 여부 체크. 타이틀이 아니라면 NaN
        # 곡 세부 정보를 보기 전에 타이틀 태그를 확인해야함
        # 타이틀곡은 '타이틀 곡'이라고 수집이 됩니다. 
        try:
            is_title.append(driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/div/span').text)
        except NoSuchElementException as e:
            is_title.append('')
                
        # 곡 정보로 들어가기
        try:
            driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/div/a[1]/span').click()
            
            # 제목 크롤링(제목은 없지 않기 때문에 예외 없음)
            title.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text)

            # 앨범 크롤링
            try:
                album.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a').text)
            except NoSuchElementException as e:
                album.append('')

            # 발매날짜 크롤링
            try:
                release_date.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text)
            except NoSuchElementException as e:
                release_date.append('')

            try:
                song_genre.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text)
            except NoSuchElementException as e:
                song_genre.append('')

            # 좋아요 수 크롤링(좋아요도 0인 경우만 있고 없지는 않기에 예외 없음)
            like.append(driver.find_element_by_xpath('//*[@id="d_like_count"]').text)

            # 가사 크롤링
            try:
                lyrics.append(driver.find_element_by_xpath('//*[@id="d_video_summary"]').text)
            except NoSuchElementException as e:
                lyrics.append('')

            # 작사 작곡 내역이 없는 경우
            try:
                creator.append(driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul').text)
            except NoSuchElementException as e:
                creator.append('')

            n += 1
            
            # 가수 이름 붙이기
            artist.append(names[i])
            
            # 디버깅    
            print(len(title), title[len(title)-1])

        except NoSuchElementException as e:
            i += 1
            break
        
        

4643 기분 좋은 날
4644 제발
4645 제발
4646 제발(Inst.)
4647 제발
4648 제발 (Inst.)
4649 오늘 뭐 먹지
4650 오늘 뭐 먹지 (Inst.)
4651 바람이 분다
4652 바람이 분다 (Inst.)
4653 떠나지마
4654 11시 11분
4655 11시 11분 (Inst.)
4656 기분좋은날
4657 사랑은 없다
4658 행복해라
4659 피도 눈물도 없이
4660 So Long
4661 어차피
4662 해&해
4663 Forget About Love
4664 사랑은 유치하게
4665 방랑자
4666 Who-Who
4667 유일무이
4668 미안해
4669 행복해라 (Remix)
4670 No Time To Lose
4671 You
4672 막상막하
4673 시간은 가도...
4674 미안해
4675 Why
4676 슬픈 약속
4677 인연
4678 Sexy Lady
4679 Deep Down
4680 Window Pain
4681 Rose
4682 난 단지 그대가
4683 초록비
4684 Tulip
4685 사랑해
4686 너의 집 앞에서
4687 Conflict In Me
4688 My Girl
4689 지킬수 없는 약속
4690 너의 향기를 느끼며
4691 오! 사랑
4692 초록비 (Inst.)
4693 다시
4694 Boy Club
4695 The Guy
4696 Always
4697 Mam
4698 Night
4699 Beautiful Day
4700 추억의 끝에 서서
4701 첫눈에 반한 사랑
4702 이젠
4703 Love Takes Time
4704 Love Takes Time (Inst.)
4705 돌아와줘요
4706 돌아와줘요 (Inst.)
4707 My Valentine
4708 My Valentine (Inst.)
4709 Lie
4710 Lie (Inst.)
4711 Because Of You
4712 Because Of You (Inst.)
4713 심장에 쥐가나서
4714 Still Be

5119 Lick It Up
5120 Rock and Roll All Nite
5121 Deuce
5122 Love Gun
5123 Fits Like a Glove
5124 Heaven`s on Fire
5125 Cold Gin
5126 Black Diamond
5127 Bang Bang You
5128 No No No
5129 Firehouse
5130 Crazy Crazy Nights
5131 Calling Dr. Love
5132 War Machine
5133 Reason to Live
5134 Tears Are Falling
5135 I Love It Loud
5136 Strutter
5137 Shout It out Loud
5138 Lick It Up
5139 Rock and Roll All Nite
5140 Detroit Rock City
5141 Laye
5142 Creatures Of The Night (From `The Magic Strings Of Frankie Presto: The Musical Companion`)
5143 夢の浮世に笑いてみな / Yumeno Ukiyoni Saitemina (꿈의 속세에 피어보아라)
5144 Rock And Roll All Nite
5145 SAMURAI SON
5146 夢の浮世に笑いてみな / Yumeno Ukiyoni Saitemina (꿈의 속세에 피어보아라) (Off Vocal Ver.)
5147 Rock And Roll All Nite (Off Vocal Ver.)
5148 SAMURAI SON (Off Vocal Ver.)
5149 夢の浮世に笑いてみな / Yumeno Ukiyoni Saitemina (꿈의 속세에 피어보아라)
5150 Rock And Roll All Nite
5151 夢の浮世に笑いてみな / Yumeno Ukiyoni Saitemina (꿈의 속세에 피어보아라) (Off Vocal Ver.)
5152 Rock And Roll All Nite (Off Vocal Ver.)
5153 I

5382 Psycho Circus (Live At B.C. Place Stadium, Vancouver/1999)
5383 Shout It Out Loud (Live At B.C. Place Stadium, Vancouver/1999)
5384 Deuce (Live At B.C. Place Stadium, Vancouver/1999)
5385 Heaven`s On Fire (Live At B.C. Place Stadium, Vancouver/1999)
5386 Into The Void (Live At B.C. Place Stadium, Vancouver/1999)
5387 Firehouse (Live At B.C. Place Stadium, Vancouver/1999)
5388 Do You Love Me (Live At B.C. Place Stadium, Vancouver/1999)
5389 Let Me Go Rock `N` Roll (Live At B.C. Place Stadium, Vancouver/1999)
5390 I Love It Loud (Live At B.C. Place Stadium, Vancouver/1999)
5391 Lick It Up (Live At B.C. Place Stadium, Vancouver/1999)
5392 100,000 Years (Live At B.C. Place Stadium, Vancouver/1999)
5393 Love Gun (Live At B.C. Place Stadium, Vancouver/1999)
5394 Black Diamond (Live At B.C. Place Stadium, Vancouver/1999)
5395 Beth (Live At B.C. Place Stadium, Vancouver/1999)
5396 Rock And Roll All Nite (Live At B.C. Place Stadium, Vancouver/1999)
5397 Strutter
5398 Nothin' To Lose
5399 F

5693 C`mon And Love Me
5694 Anything For My Baby
5695 She
5696 Love Her All I Can
5697 Rock And Roll All Nite
5698 Detroit Rock City
5699 King Of The Night Time World
5700 Ladies Room
5701 Makin` Love
5702 Love Gun
5703 Calling Dr. Love
5704 Christine Sixteen
5705 Shock Me
5706 Hard Luck Woman
5707 Tomorrow And Tonight
5708 I Stole Your Love
5709 Beth
5710 God Of Thunder
5711 I Want You
5712 Shout It Out Loud
5713 All American Man
5714 Rockin` In The U.S.A.
5715 Larger Than Life
5716 Rocket Ride
5717 Any Way You Want It
5718 Plaster Caster
5719 God Gave Rock 'N' Roll To You II (Album Version)
5720 Detroit Rock City (Edit)
5721 Black Diamond (Album Version)
5722 Hard Luck Woman (Album Version)
5723 Sure Know Something
5724 Love Gun (Album Version)
5725 Deuce (Album Version)
5726 Goin' Blind
5727 Shock Me (Album Version)
5728 Do You Love Me
5729 She (Album Version)
5730 I Was Made For Lovin' You (Album Version)
5731 Shout It Out Loud (Live (1996))
5732 God Of Thunder (Album Version)
5733

6075 Suddenly
6076 착각
6077 친구였던 너
6078 Be My Love
6079 변명
6080 The Only One
6081 상심
6082 Vacation
6083 Lucky Chance
6084 If
6085 찻잔
6086 In Your Eyes
6087 Paradise
6088 My Love
6089 하얀 그리움
6090 Epilogue
6091 Prologue
6092 Never
6093 다시 친구로
6094 Remember
6095 Outside
6096 My Girl
6097 한(恨)
6098 착한 그녀
6099 변심
6100 TV Love
6101 Non-Stop
6102 거짓말
6103 I Love You
6104 별
6105 Lucky Chance
6106 Thank You
6107 반지
6108 Thank You (Inst.)
6109 반지 (Inst.)
6110 알면서도
6111 Shadow Dance (Feat. 은지원)
6112 I`m Sorry
6113 Merry Christmas
6114 나야
6115 그게 너이길
6116 그 날 이후
6117 우린 지금 헤어지지만... (Merry Christmas Piano Ver. - With 정태, 지원)
6118 Happy Again
6119 Good Bye
6120 한마디
6121 Shadow Dance (Acoustic Remix)
6122 원해
6123 Run Away Lover
6124 Be With You
6125 널 바라보며 (Good Bye Piano Ver. - With 해영, 현수)
6126 알면서도 (Acoustic Remix)
6127 벙어리 사랑
6128 Alone
6129 Girl
6130 선물
6131 반지
6132 C`est La Vie
6133 Someone To Love
6134 휘파람 (Remake Ver.)
6135 Crazy For You
6136 Beyond The Time
6137 로망스
6138 Girl
6139 Girl
6140 남

...

16802 그녀는 바람둥이야 (Feat. 동우 Of 인피니트) (Babysoul&Kei)
16803 Delight (Yoo Ji-Ae Solo)
16804 너만 없다 (JIN Solo)
16805 어머나
16806 난 사랑을 아직 몰라
16807 사랑 사랑 누가 말했나 & 꿈을 먹는 젊은이
16808 영동부르스
16809 왜요 (Why?)
16810 왜요 (Why?) (Inst.)
16811 손들어 (Hands Up)
16812 손들어 (Hands Up) (Inst.)
16813 전체 차렷 (Attention)
16814 전체 차렷 (Attention) (Inst.)
16815 My Type
16816 My Type (Inst.)
16817 Prequel
16818 I (knew it)
16819 So Good
16820 금요일밤
16821 자꾸자꾸
16822 자꾸자꾸
16823 내 맘대로 해
16824 나 너 좋아해?
16825 오렌지 카푸치노
16826 Talk About U
16827 First Kiss
16828 First Kiss (Inst.)
16829 CLOSER
16830 센스넘쳐 (Sense Man)
16831 B.F
16832 넘나 좋은 것 (I Like U Too Much)
16833 SUGAR BABY
16834 I DO LOVE YOU
16835 CUSHION
16836 빙그르르
16837 깊어
16838 OK
16839 다 거짓말
16840 상영시간 무한대
16841 Into me (Intro)
16842 Deja Vu
16843 Love Call
16844 국민 여동생
16845 가는거야
16846 아낌없이 주는 나무
16847 골목길
16848 이기적인 걸
16849 Make It Rain
16850 That`s Right
16851 타이트하게
16852 숨은 그림 찾기
16853 이기적인 걸
16854 품행제로
16855 찰리채플린
16856 도둑 (Thief)
16857 Nobody But You
16858 배째 (Feat. 인

17262 Yes or No
17263 기적을 바란다 (I wish a miracle)
17264 보일 듯 말 듯 (Cherish)
17265 BRAVO!
17266 여기여기 붙어라 (Catch me!)
17267 나두 (dito)
17268 불러 (call me)
17269 Holic
17270 Party2nite
17271 TENSION UP!
17272 위험해 (SO, DANGEROUS)
17273 그대로 (Come as you are)
17274 Never Ending
17275 Phoenix
17276 Come with me
17277 Dash
17278 Spotlight (光)
17279 예쁘다고 (Gorgeous)
17280 Give It To Me
17281 She`s Mine
17282 ABC (Middle of The Night)
17283 Flower (You)
17284 비너스 (Dance With Me)
17285 Winter Breeze
17286 She`s Mine
17287 She`s Mine (Inst.)
17288 ABC (Middle of the Night)
17289 ABC (Middle of the Night) (Inst.)
17290 Flower (You)
17291 Flower (You) (Inst.)
17292 비너스 (Dance With Me)
17293 비너스 (Dance With Me) (Inst.)
17294 겨울잠 (Here I am)
17295 겨울잠 (Here I am) (Inst.)
17296 No Doubt (노답)
17297 신기루 (Mirrage)
17298 Brotherhood
17299 소문내지마
17300 No Doubt (노답) (Inst.)
17301 Brotherhood
17302 소문내지마
17303 Brotherhood (Inst.)
17304 소문내지마 (Inst.)
17305 Under The Moonlight (달빛 아래서)
17306 Good Bye
17307 신세계 (Long

17683 스릴러
17684 조아? 조아!
17685 우우 (WooWoo)
17686 어른
17687 Pick up the phone
17688 데리러 와 (TAKE ME)
17689 Sweet Dream
17690 Blue Day
17691 우우 (WooWoo) (Inst.)
17692 나랑 사귈래 (투제니 삽입곡 by 코코아)
17693 굿밤 (Good Night)
17694 Eye Contact
17695 시월에 설악산 (예빈&솜이)
17696 듣고싶어 (E905)
17697 굿밤 (Good Night) (Inst.)
17698 듣고싶어 (E905)
17699 넌 달라
17700 여.사.친 (우리사이)
17701 LO OK (빈챈현스S Ver.)
17702 키스해줘 (빈챈현스S Ver.)
17703 니맘대로야 (빈챈현스S Ver.)
17704 DARLING MY SUGAR (L.U.B Ver.)
17705 한입만 (L.U.B Ver.)
17706 I need healing (L.U.B Ver.)
17707 #GMGN (Good morning & Good night) (Limited Ver.)
17708 파라다이스 (Limited Ver.)
17709 듣고싶어 (E905) (Inst.)
17710 남자는 여자를 귀찮게 해
17711 나랑 사귈래
17712 남.사.친
17713 사월 (Feat. 딘딘)
17714 마네킹
17715 꽃,달,술
17716 빛
17717 이 노래 들어볼래
17718 너만 모르나 봄
17719 乾坤坎離
17720 나랑 사귈래 2016
17721 꽃,달,술
17722 나랑 사귈래 (Ballad Ver.)
17723 나랑 사귈래 (Inst.)
17724 꽃, 달, 술
17725 7과 4분의3 (널+만나러+가는+길)
17726 Mr. Potter
17727 꽃, 바람 그리고 너 (DIA Ver.)
17728 #더럽 (The love...)
17729 Mr. Potter (Inst.)
17730 #더럽 (The love...) (Inst.

18121 Juliet
18122 Cherish
18123 E (Unit Yellow Ver.)
18124 E
18125 24
18126 Rising Star
18127 비가 내리면
18128 비가 내리면 (Inst.)
18129 하늘 아래서
18130 내 곁에 서 있어줘
18131 내 곁에 서 있어줘 (Inst.)
18132 오월애 (俉月哀)
18133 오월애 (俉月哀) (Inst.)
18134 Celebrate
18135 Celebrate
18136 Celebrate (Inst.)
18137 나를 기억해
18138 사랑하기 때문에
18139 HAVE A GOOD NIGHT (Stage Ver.)
18140 뒤돌지마
18141 TIMELINE
18142 나를 기억해 (Acoustic Ver.)
18143 말도 안돼
18144 뺏길까봐
18145 느린 이별
18146 FLOWER
18147 LIGHT
18148 오아시스
18149 EYEZ EYEZ
18150 IN THE AIR
18151 얼타
18152 이 나쁜..
18153 SUNRISE
18154 What time is it now?
18155 아무렇지 않은 척
18156 Beautiful
18157 날 보며 웃어준다
18158 The Chemistry
18159 #떨려 (VICTON Ver.)
18160 #떨려
18161 Mi Amor
18162 정전기
18163 Mi Amor (Inst.)
18164 절묘해
18165 절묘해 (Inst.)
18166 다크서클
18167 다크서클 (Inst.)
18168 Dark Circle
18169 Dark Circle (Inst.)
18170 잘 부탁드립니다
18171 꿈꾸는 마음으로
18172 호두까기 인형
18173 르네상스
18174 설레는 밤
18175 겨울잠
18176 꿈꾸는 마음으로 (Chinese Ver.)
18177 크리스마스데이 (Christmas Day)
18178 KISS ME (키스 미)
18179 HAPPY
18180 기적 같은 아이 (Mir

18574 Tomorrow (CD only)
18575 Right Here
18576 L.O.U
18577 지킬게 (KeePer) (Prod. 박경)
18578 지킬게 (KeePer) (Prod. 박경)
18579 The Start
18580 Giddy Up
18581 Text Me Back
18582 Just U
18583 Back 2 U
18584 Get It
18585 Intro
18586 소년 (Boy)
18587 시간이 안 지나가 (Walkin` In Time)
18588 있어 (Got It)
18589 I`m Your Boy
18590 소년 (Boy) (Inst.)
18591 어른이 된다면
18592 어른이 된다면 (Inst.)
18593 우리끼리
18594 우리끼리 (Inst.)
18595 너는 달 지구
18596 13월 32일
18597 벚꽃피는 날에
18598 벚꽃피는 날에 (Acoustic Ver.)
18599 벚꽃피는 날에 (Inst.)
18600 Ooh Ooh
18601 Ooh Ooh (Inst.)
18602 미친듯이 (Girl Gang)
18603 미친듯이 (Girl Gang) (Inst.)
18604 우리가 함께한 시간
18605 우리가 함께한 시간 (Inst.)
18606 Love & Evil
18607 알 수 없는 비밀 (Sonatine)
18608 비의 목소리 51db (Rain 51db)
18609 지금, 좋아해 (Love & Live)
18610 You and Me Together
18611 Fairy Tale
18612 3월을 기다려 (Valentine Girl)
18613 Into the New Heart (Guitar By 김정모 of TRAX)
18614 지금, 좋아해 (Love & Live)
18615 You and Me Together
18616 Fairy Tale
18617 3월을 기다려 (Valentine Girl)
18618 지금, 좋아해 (Love & Live) (Remix) (CD Only)
18619 Yo

## Dataframe에 크롤링된 데이터 저장하기

In [180]:
test_list = [title, album, release_date, song_genre, is_title, like, creator, lyrics]
column_names = ['title', 'artist', 'album', 'release_date', 'song_genre', 'is_title', 'like', 'creator', 'lyrics']
column_v = [title, artist, album, release_date, song_genre, is_title, like, creator, lyrics]

test_df = pd.DataFrame(column_v)

# 이렇게 집어넣었더니 행이 9개인 데이터프레임이 나와버립니다. 행렬을 바꾸어줍니다. 

In [181]:
test_p = test_df.head(9).T
test_p.head()

,0,1,2,3,4,5,6,7,8
0,태지 보이스,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,269,서태지\n작사\n서태지\n작곡\n서태지\n편곡,갈수 없는 곳이란 우리에겐 없어\n보이는 길밖에도 세상은 있어\n언제나 식지않는 마...
1,시대유감(時代遺憾),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,타이틀 곡,"3,256",서태지\n작사\n서태지\n작곡\n서태지\n편곡,왜 기다려 왔잖아\n모든 삶을 포기하는 소리를\n이 세상이 모두 미쳐버릴\n일이 벌...
2,수시아(誰是我),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,488,서태지\n작사\n서태지\n작곡\n서태지\n편곡,이 세상 그 누구도 나를 닮을 순 없네\n날 세상에 알릴 거야\n나 역시 그 누구를...
3,이 밤이 깊어 가지만 (Remix),서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,700,양현석\n작사\n서태지\n작곡\n서태지\n편곡,옛 생각에 까페 문을열고\n지난 추억을 기억하려했지\n부드러운 음악소리마저\n내마음...
4,Free Style,서태지와 아이들,시대유감(時代遺憾),1996.06.07,Rock,,599,서태지\n작사\n김종서\n작사\n서태지\n작곡\n김종서\n작곡\n서태지\n편곡,항상 난 지나치기만 했네\n이젠 난 미련없이 끝내\n됐어 찾아냈어 난\n모두가 또 ...


In [190]:
test_p.columns = column_names
test_p.tail(350)

,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics
18816,Awaken,스트레이 키즈,I am NOT,2018.03.26,Rap / Hip-hop,,"2,240",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,불편한 속\nThere's something wrong\n처음으로 나를 봤었어\n너...
18817,ROCK (돌),스트레이 키즈,I am NOT,2018.03.26,Rap / Hip-hop,타이틀 곡,"2,213",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,Yeah yeah You know me\nNo no no that's not me ...
18818,잘 하고 있어,스트레이 키즈,I am NOT,2018.03.26,Rap / Hip-hop,타이틀 곡,"3,532",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,Woah Eyo Woah Oh Oh\nWoah Eyo Woah\n잠시 넘어졌지 괜찮...
18819,3rd Eye,스트레이 키즈,I am NOT,2018.03.26,Rap / Hip-hop,,"2,166",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,눈을 감아 보이는 3rd Eye\n원하는 걸 보여줘 3rd Eye\nI just w...
18820,Hellevator,스트레이 키즈,Mixtape,2018.01.08,Electronica,,"7,390",아르마딜로\n작사\n방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한...,내 삶은 밑바닥 낭떠러지\n어두운 터널 속을 걷고 있어\n내게 잔인하고 두려운 하루...
18821,Grrr 총량의 법칙,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,,"3,634",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,총량의 법칙\nGrrr 정신은 지저분해\n난폭한 개마냥 누구 앞에\n서 있든지 상관...
18822,어린 날개,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,,"3,910",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,어리게 보이기 싫어서\n괜히 짓궂은 옷만 계속 바꿔 입고\n어른스럽고만 싶어서\n괜...
18823,YAYAYA,스트레이 키즈,Mixtape,2018.01.08,Rap / Hip-hop,,"2,874",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,Stray Kids\nWatch out\n2018\n야야 야야 야야 야야 야야 야야...
18824,GLOW,스트레이 키즈,Mixtape,2018.01.08,Ballad,인기 곡,"2,830",창빈 (3RACHA)\n작사\n필릭스\n작사\n리노\n작사\n방찬 (3RACHA)\...,쉬어도 되지만 아직 일러\n쉬어야 하지만 아직 멀어\n우리가 가야 할 길\n끝이 안...
18825,School Life,스트레이 키즈,Mixtape,2018.01.08,Dance,,"2,671",방찬 (3RACHA)\n작사\n창빈 (3RACHA)\n작사\n한 (3RACHA)\n...,아침 7시 알람이 댕\n창문 사이 햇빛은 쨍\n일어나란 엄마의 잔소리는\n마치 한여...


In [183]:
test_p.shape

(19166, 9)

In [188]:
# 간혹 이상하게 수집된 곡들이 많습니다. 이런 곡들은 전처리에서 해결해줍시다.
test_p['title'].isnull().sum()

333

In [184]:
test_p.to_csv('song_data.csv')

### JTL 이후로 알 수 없는 오류가 발생했기 때문에 앞의 50명을 날리고 재시작하기 위한 전처리

In [176]:
# idol_list = idol_list.drop(idol_list.index[0:51]).reset_index(drop=True)
# idol_list.head()

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id
0,비오케이,B.O.K.,4,이정호,남,NaN,1050192
1,엑스라지,X-Large,4,고재형,남,상마인드,100779
2,보이 클럽,Boy Club,3,NaN,남,NaN,101123
3,오션,5tion,5,오병진,남,신나라 뮤직,101223
4,플래티넘,PLT,5,NaN,남,DR 뮤직,100765
